In [204]:
import numpy as np
import tensorflow as tf
from data_gen import *
import random
import matplotlib.pyplot as plt
import numpy as np
import math
#gloabal_seed =  round(random.random()*100000)
gloabal_seed =  67121
print ("seed", gloabal_seed)

seed 67121


In [209]:
num_features = 4
batch_size = 1
state_size = 2
datatype = tf.float64
num_of_operations = 3



np.random.seed(gloabal_seed)
tf.set_random_seed(gloabal_seed)
sess = tf.InteractiveSession()

x,y = samples_generator(np_center, (batch_size, num_features) , (-100,100), gloabal_seed)

batchX = tf.Variable(x, dtype=datatype)
state  = tf.Variable(np.random.rand(batch_size, state_size), dtype=datatype)
W      = tf.Variable(np.random.rand(state_size+num_features, state_size), dtype=datatype, name="W")
W2     = tf.Variable(np.random.rand(state_size, num_of_operations),dtype=datatype, name="W2")
W3     = tf.Variable(np.random.rand(num_of_operations, num_features),dtype=datatype, name="W3")

batchX.initializer.run()
state.initializer.run()
W.initializer.run()
W2.initializer.run()
W3.initializer.run()

In [210]:
list(zip(x, y))

[(array([-63.75753726,  99.2127466 ,   7.23473607, -90.12625023]),
  array([ -51.89846105,  111.0718228 ,   19.09381227,  -78.26717402]))]

In [132]:

def custom_softmax(x, base = 1):
    maxx = tf.reduce_max(x, axis=1, keep_dims=True)
    print("maxx")
    print(maxx.eval())
    maxg = (x - maxx)
    print("maxg")
    print(maxg.eval())
    #powx = tf.pow(maxg, tf.cast(-base, datatype))
    maxg =tf.nn.relu(maxg)
    powx = maxg
    for i in range(base):
        powx = tf.sqrt(powx)
    #reduced  = tf.reduce_sum(powx, axis=1, keep_dims=True)

    return powx
#tf.maximum


def tf_softmax(x):
    sum_x = tf.reduce_sum(tf.exp(x))
    print("sum_x")
    print(sum_x.eval())
    soft_x = tf.exp(x)/sum_x
    print("soft_x")
    print(soft_x.eval())
    return soft_x
           
'''
def custom_softmax( x,base = 380):
        maxx = tf.reduce_max(x, axis=1, keep_dims=True)
        powx = tf.pow(tf.cast(base, datatype), x-maxx)
        reduced  = tf.reduce_sum(powx, axis=1, keep_dims=True)
        maxg = powx/ reduced
        return maxg
'''

def calc_gradW3(W3, softmax):
    shape = W3.get_shape().as_list()
    der = tf.Variable(np.zeros(shape), dtype=datatype)
    der.initializer.run()
    for i in range(shape[0]):
        for j in range(shape[1]):
            d_temp = tf.Variable(np.zeros(shape), dtype=datatype)
            d_temp.initializer.run()
            tf.assign( d_temp[i,j], 1).eval()
            der_temp = tf.reduce_sum(- tf.matmul(softmax, d_temp))
            tf.assign( der[i,j], der_temp).eval()
    return der

In [5]:
def run_pass(batchX, state):
    X_and_state_con = tf.concat([batchX, state], 1)
    print("X_and_state_con")
    print(X_and_state_con.eval())
    layer1          = tf.matmul(X_and_state_con, W)
    print("layer1")
    print(layer1.eval())
    #next_state      = tf.nn.sigmoid(layer1)
    next_state      = tf.nn.relu(layer1)
    print("next_state")
    print(next_state.eval())
    state_dropped = tf.layers.dropout(next_state, 0, training = True)
    #print("state_dropped")
    #print(state_dropped.eval())
    logits = tf.matmul(state_dropped, W2)
    print("logits")
    print(logits.eval())
    #softmax = custom_softmax(logits, 1)
    softmax = tf.nn.softmax(logits)
    #print("cust_softmax")
    #print(cust_softmax.eval())
    print("softmax")
    print(softmax.eval())
    #softmax_loss = skewed_sigmoid_der_tf(softmax, num_of_operations)
    #print("softmax_l2_losts")
    #print(softmax_loss.eval())
    next_x = tf.matmul(softmax, W3)
    print("next_x")
    print(next_x.eval())
    selection = tf.multinomial(softmax, 1)
    #reshape = tf.reshape(selection , [batch_size, -1])
    print("selection")
    sel  = selection.eval() 
    print(sel)
    
    label = tf.one_hot(sel, num_of_operations, dtype=datatype)
    print("label")
    print(label.eval())    
    loss = tf.nn.l2_loss(label - softmax)
    print("loss")
    print(loss.eval())
    #grads  = tf.gradients(loss, [W,W2,W3])
    print("tf_discounted_epr")
    tf_discounted_epr = tf.cast( sel * 0.0, datatype)
    print(tf_discounted_epr.eval())
    optimizer = tf.train.RMSPropOptimizer(0.01)
    tf_grads = optimizer.compute_gradients(loss, var_list= [W,W2])
    tf_grads_dics = optimizer.compute_gradients(loss, var_list= [W,W2], grad_loss=tf_discounted_epr)
    train_op = optimizer.apply_gradients(tf_grads)
    print(tf_grads)
    print("weights")
    [print(w.eval()) for w in [W,W2]]
    print("grads_tf")
    [print(grad[0].eval()) for grad in tf_grads]
    print("grads_tf_disc")
    [print(grad[0].eval()) for grad in tf_grads_dics]
    print("affter applying op")
    
    [print(w.eval()) for w in [W,W2]]
    #loss fn
    #softmax = e^x/sum(e^x)
    #loss = reduced (y - softmax(relu([x;state_prev] * W1) * W2) * W3)
    #print("dev_loss_w3")
    #h = 0.001
    #dev_loss_w3 = calc_gradW3(W3, softmax)
    #print(dev_loss_w3.eval())
    return next_x, next_state

In [6]:
state1 = run_pass(batchX, state)[1].eval()

X_and_state_con
[[-0.62286753  0.18856623  0.30670629]]
layer1
[[-0.00982747 -0.09161487]]
next_state
[[ 0.  0.]]
logits
[[ 0.  0.]]
softmax
[[ 0.5  0.5]]
next_x
[[ 0.70501444]]
selection
[[1]]
label
[[[ 0.  1.]]]
loss
0.25
tf_discounted_epr
[[ 0.]]
[(<tf.Tensor 'gradients/MatMul_grad/tuple/control_dependency_1:0' shape=(3, 2) dtype=float64>, <tf.Variable 'W:0' shape=(3, 2) dtype=float64_ref>), (<tf.Tensor 'gradients/MatMul_1_grad/tuple/control_dependency_1:0' shape=(2, 2) dtype=float64>, <tf.Variable 'W2:0' shape=(2, 2) dtype=float64_ref>)]
weights
[[ 0.3751096   0.30023626]
 [ 0.27499064  0.26933108]
 [ 0.56067377  0.1454349 ]]
[[ 0.74150469  0.09561272]
 [ 0.52968019  0.34472194]]
grads_tf
[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
[[ 0.  0.]
 [ 0.  0.]]
grads_tf_disc
[[ 0.  0.]
 [ 0.  0.]
 [ 0.  0.]]
[[ 0.  0.]
 [ 0.  0.]]
affter applying op
[[ 0.3751096   0.30023626]
 [ 0.27499064  0.26933108]
 [ 0.56067377  0.1454349 ]]
[[ 0.74150469  0.09561272]
 [ 0.52968019  0.34472194]]


In [ ]:
state2 = run_pass(batchX, state1)[1].eval()

In [ ]:
state3 = run_pass(batchX, state2)[1].eval()

In [ ]:
#use derivative of sigmoid to penelise for values other than 1 or 0
def sigmoid_der(x, scale = 10):
    shifted_x = x - 0.5
    scaled_x = scale*shifted_x
    res = tf.sigmoid(scaled_x)*(1 - tf.sigmoid(scaled_x))
    return (res - 0.01)*scale**3

In [ ]:
def skewed_dist(x, scale=10):
    mean = 0.5
    str_d = mean/3.5
    const = 1/tf.sqrt(2*math.pi*(str_d**2))
    power = ((x-mean)**2) / (2*str_d**2)
    return const*math.exp(-power)*scale**2

In [ ]:
def skewed_sigmoid_der(x, scale = 10, num_ops = 3):
    shifted_x = x - 0.5
    scaled_x = scale*shifted_x
    nom = tf.exp(-scaled_x)+num_ops*tf.exp(-num_ops*scaled_x)+(num_ops+1)*tf.exp(-(num_ops+1)*scaled_x)
    denom = tf.square(tf.exp(-scaled_x) + 1)*tf.square(tf.exp(-num_ops*scaled_x) + 1)
    res = nom/denom
    return (res - 0.01)*scale**3

def sigmoid(x, scale = 10, num_ops = 3):
    shifted_x = x - 0.5
    scaled_x = scale*shifted_x
    return tf.sigmoid(scaled_x)*tf.sigmoid(10*scaled_x)

In [53]:
def plot(fn, ops, subl):
    t = np.arange(0.0, 1.01, 0.01)
    s = [fn(val, ops) for val in t]
    max_val = max(s)
    probs = [1 - pen/max_val for pen in s]
    fig, ax1 = plt.subplot(*subl)
    ax2 = ax1.twinx()
    
    ax1.plot(t, s, "blue")
    ax1.xlabel('softmax')
    ax1.ylabel('penalty')
    
    ax2.plot(t, probs, "red")
    ax2.ylabel('Probability')

    fig.title("Skewed sigmoid der per "+str(ops)+ "ops")
    fig.grid(True)


In [10]:
def skewed_sigmoid_der_py(x, num_ops = 3, scale = 10):
    worst_case = 1 / num_ops
    shifted_x = x - worst_case + 0.02
    scaled_x = scale*shifted_x
    nom = scale*math.exp(-scaled_x)+scale*num_ops*math.exp(-num_ops*scaled_x)
    #denom = math.square(math.exp(-scaled_x) + 1)*math.square(math.exp(-(num_ops-1)*scaled_x) + 1)
    denom = ((math.exp(-scaled_x) + 1)**2) * ((math.exp(-(num_ops-1)*scaled_x) + 1)**2)
    res = nom/denom
    return worst_case*((res)*scale**3)

def skewed_sigmoid_der_tf(x, num_ops = 3, scale = 10):
    worst_case = 1 / num_ops
    shifted_x = x - worst_case + 0.02
    scaled_x = scale*shifted_x
    nom = scale*tf.exp(-scaled_x)+scale*num_ops*tf.exp(-num_ops*scaled_x)
    denom = tf.square(tf.exp(-scaled_x) + 1)*tf.square(tf.exp(-(num_ops-1)*scaled_x) + 1)
    res = nom/denom
    return worst_case*((res - 0.01)*scale**3)

In [54]:
plt.figure(figsize=(15,7))
for ops in range(6):
    plot(skewed_sigmoid_der_py, ops+2, (2,3, ops+1))
plt.subplots_adjust(top=0.92, bottom=0.08, left=0.10, right=0.95, hspace=0.30,
                    wspace=0.20)
plt.show()

TypeError: 'AxesSubplot' object is not iterable

In [ ]:
plot(skewed_sigmoid_der_tf, 2)
plot(skewed_sigmoid_der_tf, 3)
plot(skewed_sigmoid_der_tf, 4)
plot(skewed_sigmoid_der_tf, 5)
plot(skewed_sigmoid_der_tf, 6)

In [ ]:
t = np.arange(0.0, 1.01, 0.01)
for val in t:
    print (val, skewed_sigmoid_der_py(val, 5, 10))

In [ ]:
import math

def sigmoid(x):
    return 1 / (1 + math.exp(-x))

def sig_inv_loss(max_error_tot):
    return (1 - sigmoid( (max_error_tot/50) - 10))

def sig_inv_loss_tf(max_error_tot):
    return (1 - tf.sigmoid( (max_error_tot/50) - 10)).eval()*0.5

def log_loss_tf(max_error_tot):
    return tf.log1p(max_error_tot*100).eval()


In [ ]:
t = np.arange(0, 100000, 100.0)
for val in t:
    print (val, log_loss_tf(val))

In [118]:
def gradtest2(batchX, state):
    X_and_state_con = tf.concat([batchX, state], 1)
    print("X_and_state_con")
    print(X_and_state_con.eval())
    layer1          = tf.matmul(X_and_state_con, W)
    print("layer1")
    print(layer1.eval())
    next_state      = tf.nn.sigmoid(layer1)
    print("next_state")
    print(next_state.eval())
    logits = tf.matmul(next_state, W2)
    print("logits")
    print(logits.eval())
    #softmax = tf_softmax(logits)
    softmax = tf.nn.softmax(logits)
    #print("cust_softmax")
    #print(cust_softmax.eval())
    print("softmax")
    print(softmax.eval())
    #print("softmaxtf")
    #print(softmaxtf.eval())
    
    grads  = tf.gradients(softmax, logits)
    alhpa_jacobean = calc_softmax_jacob(logits)
    print("weights2")
    [print(w.eval()) for w in [W2]]
    print("grads")
    [print(grad.eval()) for grad in grads]
    print("alhpa_jacobean")
    print(alhpa_jacobean.eval())

In [127]:
def single_softmax(val, x):
    #print("single_softmax")
    sum_x = tf.reduce_sum(tf.exp(x))
    #print("sum_x")
    #print(sum_x.eval())
    soft_val = tf.exp(val)/sum_x
    #print("soft_val")
    #print(soft_val.eval())
    return soft_val

In [128]:
def calc_softmax_jacob(alpha):
    leng = alpha.get_shape().as_list()[1]
    shape=(leng,leng)
    jacobian = tf.Variable(np.zeros(shape), dtype=datatype)
    jacobian.initializer.run()
    #print("calc_softmax_jacob")
    #print("aplha")
    #print(alpha.eval())
    for i in range(shape[0]):
        for j in range(shape[1]):
            i_val = alpha[0][i].eval()
            j_val = alpha[0][j].eval()
            #print(i, j, i_val, j_val)
            if i == j:
                der = single_softmax(i_val,alpha) * (1 - single_softmax(j_val,alpha))
            else:
                der = -single_softmax(i_val,alpha) *  single_softmax(j_val,alpha)
            tf.assign( jacobian[i,j], der.eval()).eval()
    return jacobian

In [135]:
gradtest2(batchX, state)

X_and_state_con
[[ 0.65224106  0.82612053  0.32550767]]
layer1
[[ 0.82109083  0.47648064]]
next_state
[[ 0.69446784  0.61691649]]
logits
[[ 0.29210583  0.80191096  0.25311898]]
softmax
[[ 0.27573047  0.45908206  0.26518747]]
weights2
[[ 0.14580228  0.85268734  0.21834923]
 [ 0.30936251  0.33999257  0.16449952]]
grads
[[ 0.  0.  0.]]
alhpa_jacobean
[[ 0.19970318 -0.12658291 -0.07312027]
 [-0.12658291  0.24832572 -0.12174281]
 [-0.07312027 -0.12174281  0.19486307]]


In [154]:
def discount_rewards( rewards):
        """ take 1D float array of rewards and compute discounted reward """
    
        #discount the rewards
        discounted_r = np.zeros_like(rewards)
        running_add = 0
        
        #if rewards did not swith, penelise the frst selection first
        #ng = range(0, len(checked_rewards)) if rewards == checked_rewards else reversed(range(0, len(checked_rewards)))
        for t in reversed(range(0, len(rewards))):          
            running_add = running_add * 0.6 + rewards[t] #for all pos/negative rewards mean is always going to be bigger than the first reward, hence it will become positive when centered
            print(running_add, rewards[t])
            discounted_r[t] = running_add
        #normalise rewards
        
        #dont scale but norm, as scaling might result into inversion of signs
        normalised_r = (discounted_r - np.mean(discounted_r)) / (np.std(discounted_r) + 1e-10)
        #normalised_r = discounted_r/ np.linalg.norm(discounted_r, 2)
        return normalised_r, discounted_r


In [156]:
normalised_r, discounted_r = discount_rewards( [3000.0, 3000.0,    0.0,     0.0,     0.0])
print(discounted_r)
print(normalised_r)

0.0 0.0
0.0 0.0
0.0 0.0
3000.0 3000.0
4800.0 3000.0
[ 4800.  3000.     0.     0.     0.]
[ 1.62520902  0.72231512 -0.78250805 -0.78250805 -0.78250805]
